In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
imageSize = [224, 224]

trainPath = r"/content/drive/MyDrive/Nail Disease/Dataset/train"

testPath = r"/content/drive/MyDrive/Nail Disease/Dataset/test"


In [ ]:
# adding preprocessing layers to the front of vgg

vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(17, activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'], run_eagerly=True
)


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [ ]:
training_set.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [ ]:
len(training_set)

21

In [ ]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set)//3,
  validation_steps=len(test_set)//3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/100
7/7 [==============================] - 83s 9s/step - loss: 3.3197 - accuracy: 0.1161 - val_loss: 2.9436 - val_accuracy: 0.1406
Epoch 2/100
7/7 [==============================] - 37s 5s/step - loss: 3.1412 - accuracy: 0.1429 - val_loss: 3.0019 - val_accuracy: 0.1875
Epoch 3/100
7/7 [==============================] - 26s 4s/step - loss: 2.7296 - accuracy: 0.2143 - val_loss: 2.6250 - val_accuracy: 0.2812
Epoch 4/100
7/7 [==============================] - 17s 2s/step - loss: 2.4891 - accuracy: 0.2634 - val_loss: 2.9226 - val_accuracy: 0.1562
Epoch 5/100
7/7 [==============================] - 14s 2s/step - loss: 2.5686 - accuracy: 0.2812 - val_loss: 2.3067 - val_accuracy: 0.3438
Epoch 6/100
7/7 [==============================] - 11s 1s/step - loss: 2.3188 - accuracy: 0.3571 - val_loss: 2.3376 - val_accuracy: 0.2969
Epoch 7/100
7/7 [==============================] - 16s 2s/step - loss: 2.2938 - accuracy: 0.3237 - val_loss: 2.0998 - val_accuracy: 0.3594
Epoch 8/100
7/7 [==========

In [ ]:
#save the model
model.save('vgg-16-nail-disease.h5')


Test The Model

In [ ]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [ ]:
#load saved model file
model=load_model('vgg-16-nail-disease.h5')

In [ ]:
#load one random image from local system
img=image.load_img(r'19.PNG',target_size=(224,224))

In [ ]:
#convert image to array format
x=image.img_to_array(img)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[3.7915495e-06, 2.1875452e-03, 4.1685593e-07, 4.1365284e-03,
        5.5700364e-11, 7.3820658e-08, 5.7155389e-06, 1.0331855e-06,
        6.6932332e-04, 9.1808915e-01, 4.5551822e-08, 1.2603366e-03,
        2.2272835e-12, 7.7233365e-04, 7.8795410e-06, 6.9824591e-02,
        3.0412099e-03]], dtype=float32)

In [ ]:
output=np.argmax(model.predict(img_data), axis=1)


In [ ]:
index=['Darier_s disease', 'Muehrck-e_s lines', 'aloperia areata', 'beau_s lines', 'bluish nail',
       'clubbing','eczema','half and half nailes (Lindsay_s nails)','koilonychia','leukonychia',
       'onycholycis','pale nail','red lunula','splinter hemmorrage','terry_s nail','white nail','yellow nails']
result = str(index[output[0]])
result

'leukonychia'